# Analyse Croisée Météo-Tourisme
## Corrélations entre variables météo et scores

In [1]:
# EDA WEATHER-TOURISM - VERSION FONCTIONNELLE
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from pathlib import Path

# Configuration
sns.set_style("whitegrid")
plt.rcParams['figure.dpi'] = 100
pd.set_option('display.max_columns', 50)

# 1. Chargement et fusion CORRECTE des données
def load_and_merge_data():
    try:
        # Chargement avec conversion de date
        meteo = pd.read_csv("../data/processed/weather_processed.csv", parse_dates=['date'])
        tourism = pd.read_csv("../data/reports/tourism_recommendations.csv", parse_dates=['month'])
        
        # Debug: Affichage des périodes couvertes
        print(f"Météo: De {meteo['date'].min()} à {meteo['date'].max()}")
        print(f"Tourisme: De {tourism['month'].min()} à {tourism['month'].max()}")
        
        # Fusion sur ville et mois (au lieu de date exacte)
        meteo['month'] = meteo['date'].dt.to_period('M')
        tourism['month'] = tourism['month'].dt.to_period('M')
        
        df = pd.merge(
            meteo,
            tourism,
            on=['city', 'month'],
            how='inner'
        )
        
        # Conversion de Period à DateTime pour l'analyse
        df['month'] = df['month'].dt.to_timestamp()
        
        return df
    
    except Exception as e:
        print(f"Erreur: {str(e)}")
        return None

df = load_and_merge_data()

if df is None or df.empty:
    print("ERREUR: Aucune donnée après fusion. Vérifiez:")
    print("- Que les périodes se chevauchent")
    print("- Que les noms de villes correspondent")
    exit()

# 2. Analyse de base
print("\nStatistiques clés:")
print(f"Villes: {df['city'].nunique()}")
print(f"Période: {df['month'].min().date()} au {df['month'].max().date()}")
print(f"Enregistrements: {len(df)}")

# 3. Visualisations essentielles
Path("plots").mkdir(exist_ok=True)

# Relation Température-Score
plt.figure(figsize=(10, 6))
sns.regplot(
    data=df,
    x='temp',
    y='avg_score',
    scatter_kws={'alpha': 0.4, 'color': 'steelblue'},
    line_kws={'color': 'coral'}
)
plt.title("Relation Température-Score Touristique", fontsize=14)
plt.xlabel("Température Moyenne (°C)", fontsize=12)
plt.ylabel("Score Touristique", fontsize=12)
plt.savefig("plots/temp_vs_score.png", bbox_inches='tight')
plt.close()

# Scores par Saison
if 'season' in df.columns:
    plt.figure(figsize=(10, 6))
    sns.boxplot(
        data=df,
        x='season',
        y='avg_score',
        palette="viridis"
    )
    plt.title("Distribution des Scores par Saison", fontsize=14)
    plt.xlabel("")
    plt.ylabel("Score Moyen", fontsize=12)
    plt.savefig("plots/season_scores.png", bbox_inches='tight')
    plt.close()

# Top 5 meilleures conditions
if all(col in df.columns for col in ['temp', 'weather', 'avg_score']):
    top_conditions = (df.groupby('weather')
                     ['avg_score']
                     .mean()
                     .sort_values(ascending=False)
                     .head(5))
    
    print("\nTop 5 conditions optimales:")
    print(top_conditions.round(2))

print("\nAnalyse terminée avec succès!")

Météo: De 2025-06-30 11:11:37 à 2025-06-30 11:23:04
Tourisme: De 2012-10-31 00:00:00 à 2025-06-30 00:00:00

Statistiques clés:
Villes: 36
Période: 2025-06-01 au 2025-06-01
Enregistrements: 36

Top 5 conditions optimales:
weather
dust      3.00
clouds    2.53
mist      2.50
clear     2.20
rain      2.00
Name: avg_score, dtype: float64

Analyse terminée avec succès!


/tmp/ipykernel_2823/955263760.py:78: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  sns.boxplot(
